# <div align = "center">Applied Data Science Capstone, Week 1<br>Part 2: Rental Data Wrangling</div>

I have prepared two dataframes using Selenium and BeautifulSoup to scrap spacelist and poin2homes webpages. The information I obtained was the amount of available commercial rental spaces in Toronto. The data obtained has the address, area and rent cost per month. I proceded to add the coordinates and determine the neighbourhood it belongs to. The code I used can be found on my github:

'''space for github code''''

For this part I will import the data obtained to process it further:

In [1]:
import pandas as pd
df1 = pd.read_csv('toronto_rental.csv')
df2 = pd.read_csv('toronto_rental_2b.csv')

In [2]:
df1.head()

,Address,Area,Rent,Latitude,Longitude,Neighbourhood Number
0,"40 Sousa Mendes Street, Toronto ON",1000.0,4000.0,43.661313,-79.453448,93
1,"890 Dundas St W, Toronto ON",500.0,2650.0,43.651260,-79.411811,81
2,"121 - 100 Sunrise Ave, Toronto ON",3729.0,5593.0,43.717449,-79.310501,43
3,"3440 Sheppard Avenue East, Toronto ON",3400.0,16500.0,43.779482,-79.305280,118
4,"119 - 100 Sunrise Ave, Toronto ON",5131.0,7696.0,43.717492,-79.310305,43


In [3]:
df2.head()

,Address,Area,Rent,Latitude,Longitude,Neighbourhood Number
0,"1139 MORNINGSIDE AVE 37, Toronto ON",3500.00,7000.0,43.645663,-79.474971,87
1,"803 KING ST W, Toronto ON",3302.00,18986.5,43.642848,-79.407235,82
2,"1225 Kennedy Rd 2000K1, Toronto ON",392.04,2900.0,43.756179,-79.275154,126
3,"1269 DUNDAS ST W 200, Toronto ON",2500.00,7500.0,43.649259,-79.424894,84
4,"384 YONGE ST 20, Toronto ON",257.00,1388.0,43.659602,-79.382670,76


In [62]:
df_merged = df1.append(df2)

In [63]:
df_merged['Neighbourhood Number'].unique().shape

(84,)

This web scrapping on the webpage used did not turn up available commercial rental space in all neighbourhoods.

In [64]:
#Save to a dataframe the average available rental area and average rent per month on the neighbourhoods:
average_rental = df_merged.groupby('Neighbourhood Number').mean()[['Area', 'Rent']]
average_rental

,Area,Rent
Neighbourhood Number,,
0,1414.000000,6226.000000
1,1229.500000,4937.416667
2,1494.400000,4358.666667
3,1400.000000,1866.666667
17,2076.166667,5507.666667
...,...,...
133,500.000000,2000.000000
137,959.750000,2449.000000
138,1360.000000,1335.000000


In [65]:
average_rental.drop(0, axis = 0, inplace = True)

In [66]:
#Determining the average cost per month of renting a square feet in the neighbourhoods:
average_rental['CAD / ft²·month'] = average_rental['Rent']/average_rental['Area']

In [67]:
average_rental.reset_index(inplace = True) 

In [68]:
average_rental

,Neighbourhood Number,Area,Rent,CAD / ft²·month
0,1,1229.500000,4937.416667,4.015792
1,2,1494.400000,4358.666667,2.916667
2,3,1400.000000,1866.666667,1.333333
3,17,2076.166667,5507.666667,2.652806
4,19,4058.500000,1580.000000,0.389306
...,...,...,...,...
78,133,500.000000,2000.000000,4.000000
79,137,959.750000,2449.000000,2.551706
80,138,1360.000000,1335.000000,0.981618
81,139,1866.228571,3850.604762,2.063308


In [69]:
area_rent = average_rental[['Neighbourhood Number', 'CAD / ft²·month']]
area_rent

,Neighbourhood Number,CAD / ft²·month
0,1,4.015792
1,2,2.916667
2,3,1.333333
3,17,2.652806
4,19,0.389306
...,...,...
78,133,4.000000
79,137,2.551706
80,138,0.981618
81,139,2.063308


In [70]:
area_rent.iloc[:,1].describe()

count    83.000000
mean      3.165939
std       2.681352
min       0.000833
25%       2.021457
50%       2.813211
75%       3.689355
max      23.550296
Name: CAD / ft²·month, dtype: float64

Scrapping two webpages yields commercial square feet rental information of roughly over 50% of the neigbhourhoods in Toronto. With further scrapping the amount of nieghbourhoods included can increase. However; for the sake of this project, it will be determined that if a person wants to open a restaurant, roughly half of the neighbourhoods in Toronto have available commercial space for rent.

To frame the scope of this project further, no aditional analysis will be made to determine how many of the locations scrapped have suitable venues for a restaurant.

In [73]:
#Saving the neighbourhood numbers that have rental cost values, to use it to query Foursquare:
nbh_num = area_rent.iloc[:,0].to_frame()
nbh_num

,Neighbourhood Number
0,1
1,2
2,3
3,17
4,19
...,...
78,133
79,137
80,138
81,139


In [74]:
nbh_num.to_csv('ngh_num.csv', index = False)